# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [59]:
import numpy as np
import pandas as pd
import requests
import requests_cache
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [7]:
stocks = pd.read_csv('sp_500_stocks.csv')
requests_cache.install_cache(expire_after=3600)

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [8]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [9]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 95198893,
 'calculationPrice': 'close',
 'change': -4.46,
 'changePercent': -0.02598,
 'close': 160.21,
 'closeSource': 'fcilfoai',
 'closeTime': 1681822236887,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 164.39,
 'delayedPriceTime': 1678938869857,
 'extendedChange': -0.88,
 'extendedChangePercent': -0.00538,
 'extendedPrice': 159.69,
 'extendedPriceTime': 1721938008413,
 'high': 170.58,
 'highSource': 'e5iaint1c  ld dperyeume',
 'highTime': 1717708957986,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 165.32,
 'iexCloseTime': 1663628001149,
 'iexLastUpdated': 1697928233224,
 'iexMarketPercent': 0.015998009478252095,
 'iexOpen': 168,
 'iexOpenTime': 1687282738794,
 'iexRealtimePrice': 165.3,
 'iexRealtimeSize': 103,
 'iexVolume': 1459001,
 'lastTradeTime': 1683843000919,
 'latestPrice': 160.07,
 'latestSource': 'Close',
 'latestTime': 'February 23, 2022',
 'latestUpdate': 1726576453588,
 'latestVolume': 93

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [10]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [11]:
my_columns = ['Ticker', 'Stock Price', 'Market Cap', 'Number of shares to buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Cap,Number of shares to buy


In [12]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
        index = my_columns,
    ), 
    ignore_index=True
)

,Ticker,Stock Price,Market Cap,Number of shares to buy
0,AAPL,160.07,2737469461933,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [13]:
final_dataframe = pd.DataFrame(columns = my_columns)

for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series([
            stock,
            data['latestPrice'],
            data['marketCap'],
            "N/A"
        ],
        index = my_columns
        ),
        ignore_index = True
    )

In [14]:
final_dataframe

,Ticker,Stock Price,Market Cap,Number of shares to buy
0,A,160.07,2737469461933,N/A
1,AAL,160.07,2737469461933,N/A
2,AAP,160.07,2737469461933,N/A
3,AAPL,160.07,2737469461933,N/A
4,ABBV,160.07,2737469461933,N/A
...,...,...,...,...
500,YUM,160.07,2737469461933,N/A
501,ZBH,160.07,2737469461933,N/A
502,ZBRA,160.07,2737469461933,N/A
503,ZION,160.07,2737469461933,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [15]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [16]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        if symbol in data:
            final_dataframe = final_dataframe.append(
                pd.Series([
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    "N/A"
                ],
                index = my_columns
                ),
                ignore_index = True
            )

https://sandbox.iexapis.com/stable/stock/market/batch?symbols=A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA&types=quote&token=Tpk_059b97af715d417d9f49f50b51b1c448
https://sandbox.iexapis.com/stable/stock/market/batch?symbols=CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [24]:

# portfolio_size=input()
# try:
#     val = float(portfolio_size)
# except ValueError:
#     print("That's not a number!\n Please try again:")
#     portfolio_size = input('enter the value of your portfolio: ')
#     val = float(portfolio_size)

In [27]:
val = 1000000
position_size = val/len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of shares to buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
    
final_dataframe

,Ticker,Stock Price,Market Cap,Number of shares to buy
0,A,132,39385251665,15
1,AAL,16.68,11185545750,119
2,AAP,199.2,12415587081,9
3,AAPL,164.24,2708197992550,12
4,ABBV,149.59,272475701851,13
...,...,...,...,...
498,YUM,121.6,36170743700,16
499,ZBH,125.17,26509120521,15
500,ZBRA,412.63,21804736883,4
501,ZION,71.73,11009492002,27


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

- Initializing our XlsxWriter Object
- Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [58]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

background_color = '#34eb71'

header_format = writer.book.add_format({
    'bg_color': background_color,
    'bold': True
})

string_format = writer.book.add_format({
    'border': 1
})

dollar_format = writer.book.add_format({
    'num_format': '$0.00',
    'border': 1
})

num_format = writer.book.add_format({
    'num_format': '0',
    'border': 1
})

column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Cap', dollar_format],
    'D': ['Number of shares to buy', num_format]
}


for col in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{col}:{col}', 18, column_formats[col][1])
    
writer.sheets['Recommended Trades'].set_row(0, 18, header_format)

writer.save()